Script for database project 3. 

In [1]:
import pymysql 
import getpass
from IPython.display import clear_output
import datetime

In [2]:
class STUDENT:
    def __init__(self,ID,Name,Password,Address):
        self.ID = ID
        self.Name = Name
        self.Password = Password
        self.Address = Address
        
    def info(self):
        return (self.ID,self.Name,self.Password,self.Address)

In [3]:
def CheckDatabaseOpen(dbname):
    mydb = pymysql.connect("localhost","root","password")
    mycursor = mydb.cursor()
    mycursor.execute("SHOW DATABASES")
    DATABASE_EXIST = False
    for name in mycursor:
        if name[0] == dbname:
            mydb.close()
            DATABASE_EXIST = True
            return
    mydb.close()
    assert DATABASE_EXIST, "DATABASE does not exist!!!!"
    return

In [4]:
def Login(mycursor):
    while(1):
#         clear_output()
#         print("please enter your user name:")
        usr = input("please enter your user name:")
        pwd = getpass.getpass(prompt='Password: ')
        if mycursor.execute('select * from student where Name = %s and Password = %s',(usr,pwd)) > 0:
            stu_info = mycursor.fetchone()
            my_stu = STUDENT(*stu_info)
            clear_output()
            print('welcome to your student menu, %s' % usr)
            return my_stu
        else:
            clear_output()
            print("user name not exist or passowrd not correct!")


In [5]:
def EnrolledQuarter():
    now = datetime.datetime.now()
    if now.month in [9,10,11]:
        return now.year, "Q1"
    elif now.month in [12,1,2]:
        return now.year, "Q2"
    elif now.month in [3,4,5]:
        return now.year, "Q3"
    else:
        return now.year, "Q4"

def ShowStuMenu(mycursor,usr):
    year,quar = EnrolledQuarter()
    mycursor.execute('select transcript.UoSCode, unitofstudy.UoSName, \
                transcript.Year, transcript.Semester, unitofstudy.Credits\
                from student, transcript, unitofstudy\
                where student.id = transcript.Studid and \
                unitofstudy.UoSCode = transcript.UoSCode\
                and transcript.Semester = %s and transcript.Year=%s \
                and Grade is NULL and student.name = %s', (quar,year,usr) )
#     mycursor.execute('select transcript.UoSCode, unitofstudy.UoSName,\
#                 transcript.Year, transcript.Semester, unitofstudy.Credits \
#                 from student, transcript, unitofstudy \
#                 where student.id = transcript.Studid and transcript.Semester = %s \ 
#                 and transcript.Year=%s and Grade is NULL  and \
#                 unitofstudy.UoSCode = transcript.UoSCode \
#                 and student.name = %s', (quar,year,usr) )
    clear_output()
    result = mycursor.fetchall()
    print("%s is currently taking %d courses:" % (usr,len(result)))
#     print("Course NO \t Course Name \t Year \t Semester \t Credits \t Grade")
    for x in result:
        print(x[0],"\t",x[1].ljust(40),x[2],"\t",x[3],"\t",x[4])

In [6]:
def WaitForNext(current_state = 'login'):
    print('************************************************')
    if current_state == 'login':
        print('Input your option: \n Transcript \n Enroll \n Withdraw \n Personal Details \n Logout')
    elif current_state == 'Transcript':
        print('Input your option: \n Details \n Back')
    elif current_state == 'Details':
        print('Input your option: \n Back')
    elif current_state == 'Personal Details':
        print('Input your option: \n Edit \n Back')
    option = input()
    return option
    

In [7]:
def ShowTranscript(mycursor,usr):
    mycursor.execute('select transcript.UoSCode, unitofstudy.UoSName, \
                transcript.Year, transcript.Semester, unitofstudy.Credits,\
                transcript.Grade\
                from student, transcript, unitofstudy\
                where student.id = transcript.Studid and \
                unitofstudy.UoSCode = transcript.UoSCode\
                and student.name = %s\
                order by transcript.Year', (usr) )
    clear_output()
    result = mycursor.fetchall()
    print("%s's transcript is shown below " % (usr))
#     print("Course NO \t Course Name \t Year \t Semester \t Credits \t Grade")
    for x in result:
        print(x[0],"\t",x[1].ljust(40),x[2],"\t",x[3],"\t",x[4],"\t",x[5])

In [8]:
def ShowCourseDetails(mycursor,usr):
#     print("Please input the course number:")
    course_num = input("Please input the course number:")
    mycursor.execute('select transcript.UoSCode, unitofstudy.UoSName,\
            transcript.Year, transcript.Semester, \
            uosoffering.Enrollment,  uosoffering.MaxEnrollment, \
            faculty.Name, transcript.Grade\
            from  unitofstudy, uosoffering, faculty, student, transcript\
            where faculty.Id = uosoffering.InstructorId and \
            transcript.UoSCode = unitofstudy.UoSCode \
            and unitofstudy.UoSCode = uosoffering.UoSCode and \
            uosoffering.Year = transcript.Year \
            and uosoffering.Semester = transcript.Semester and \
            student.id = transcript.Studid and \
            student.name = %s and transcript.UoSCode = %s', (usr,course_num) )
#     clear_output()
    result = mycursor.fetchall()
    if len(result):
        clear_output()
        print("detailed transcript is shown below ")
        for x in result:
            print("\nCourse NO.",x[0], "\nCourse Name.",x[1],\
                 "\nYear.", x[2], "\nSemester.",x[3], "\nEnrollment",x[4],\
                 "\nMaxEnrollment.",x[5],"\nInstructor Name.",x[6], "\nGrade.",x[7])
    else:
        print("wrong course number!!!")

In [9]:
def ShowPersonalDetails(mycursor,usr):
    mycursor.execute('select * from student where student.Name = %s', (usr))
    result = mycursor.fetchall()
    clear_output()
    print("%s's personal details are shown below:" % usr)
    print("ID \t NAME \t\t PASSWORD \t ADDRESS")
    for x in result:
        print(x[0],'\t',x[1],'\t',x[2],'\t',x[3])

In [10]:
def EditPersonalDetails(mycursor,usr):
    option = input("Input your option: \n password \n address\n")
    new_content = input("Input your new content:")
    while new_content == "":
        new_content = input("Content cannot be empty. Input your new content:")
    if option == "password":
        mycursor.execute("\
                        Update student\
                        set student.Password = %s \
                        where student.Name = %s" , (new_content,usr))
    elif option == "address":
        mycursor.execute("\
                        Update student\
                        set student.Address = %s \
                        where student.Name = %s" , (new_content,usr))
    else:
        print("editing target is wrong1!!!")

In [ ]:
def main():
    dbname = 'project3-nudb'
    
    CheckDatabaseOpen(dbname)
    
    mydb = pymysql.connect("localhost","root","password",dbname)
    mycursor = mydb.cursor()
    
    states = ['waitforlogin','login','Transcript','Details','Personal Details']
    options = ['Transcript', 'Back']
    
    while(1):
    
        my_stu =Login(mycursor)
        ShowStuMenu(mycursor,my_stu.Name)
    
        current_state = 'login'
        option = WaitForNext(current_state)
    
        while option != "Logout": 
            VALID_OPTION = False
            if option == "Transcript" and current_state =='login':
                ShowTranscript(mycursor,my_stu.Name)
                current_state = 'Transcript'
                VALID_OPTION = True
            if option == "Back" and current_state == 'Transcript':
                ShowStuMenu(mycursor,my_stu.Name)
                current_state = 'login'
                VALID_OPTION = True
            if option == "Details" and current_state == 'Transcript':
                ShowCourseDetails(mycursor,my_stu.Name)
                current_state = 'Details'
                VALID_OPTION = True
            if option == "Back" and current_state == 'Details':
                ShowTranscript(mycursor,my_stu.Name)
                current_state = 'Transcript'
                VALID_OPTION = True
            if option == "Personal Details" and current_state =='login':
                ShowPersonalDetails(mycursor,my_stu.Name)
                current_state = 'Personal Details'
                VALID_OPTION = True
            if option == "Edit" and current_state =='Personal Details':
                EditPersonalDetails(mycursor,my_stu.Name)
                mydb.commit()
                ShowPersonalDetails(mycursor,my_stu.Name)
                VALID_OPTION = True
            if option == "Back" and current_state =='Personal Details':
                ShowStuMenu(mycursor,my_stu.Name)
                current_state = 'login'
                VALID_OPTION = True
            if not VALID_OPTION:
                print("wrong option !!!!!")
            option = WaitForNext(current_state)
                
        del my_stu
        clear_output()
    mycursor.close()
    mydb.close()

In [ ]:
if __name__ == "__main__":
    main()

Linda Smith's personal details are shown below:
ID 	 NAME 		 PASSWORD 	 ADDRESS
3213 	 Linda Smith 	 lunch 	 Cronulla
************************************************
Input your option: 
 Edit 
 Back


In [ ]:
# dbname = 'project3-nudb'
# mydb = pymysql.connect("localhost","root","password",dbname)
# mycursor = mydb.cursor()
# usr = "Linda Smith"
# mycursor.execute('select transcript.UoSCode, unitofstudy.UoSName, \
#                 transcript.Year, transcript.Semester, unitofstudy.Credits,\
#                 transcript.Grade\
#                 from student, transcript, unitofstudy\
#                 where student.id = transcript.Studid and \
#                 unitofstudy.UoSCode = transcript.UoSCode\
#                 and student.name = %s\
#                 order by transcript.Year', (usr) )
# result = mycursor.fetchall()
# print("%s's transcript is shown below " % (usr))
# # print("Course NO \t Course Name".ljust(50) + "\t Year \t Semester \t Credits \t Grade")
# for x in result:
#     print(x[0],"\t",x[1].ljust(40)+"\t",x[2],"\t",x[3],"\t",x[4],"\t",x[5])